In [2]:
import cv2
import numpy as np


def calculate_angle(center, point1, point2):
    vector1 = np.array([point1[0] - center[0], point1[1] - center[1]])
    vector2 = np.array([point2[0] - center[0], point2[1] - center[1]])
    angle = np.arctan2(vector2[1], vector2[0]) - np.arctan2(vector1[1], vector1[0])
    angle = np.degrees(angle)
    if angle < 0:
        angle += 360
    return angle

cap = cv2.VideoCapture(r"C:\Users\DELL\Downloads\video_humanoid.mp4")
prev_center = None
prev_point = None
frame_rate = cap.get(cv2.CAP_PROP_FPS)
angular_speed_list = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.blur(gray_frame, (3, 3))

        detected_circles = cv2.HoughCircles(gray_frame, 
                                            cv2.HOUGH_GRADIENT, dp=1, minDist=20, 
                                            param1=40, param2=32, 
                                            minRadius=23, maxRadius=33)

        if detected_circles is not None:
            detected_circles = np.uint16(np.around(detected_circles))
            
            
            # circle = detected_circles[0][0]
            # center = (circle[0], circle[1])
            # radius = circle[2]
            circle = max(detected_circles[0], key=lambda x: x[2])
            center = (circle[0], circle[1])
            radius = circle[2]
            # print(radius)

            
            point = (center[0], center[1] - radius)
            
            if prev_center is not None and prev_point is not None:
                angle = calculate_angle(center, prev_point, point)
                angular_speed = angle * frame_rate 
                angular_speed_list.append(angular_speed)
                
                
                cv2.putText(frame, f"Angular Speed: {angular_speed:.2f} deg/s", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            cv2.circle(frame, point, 3, (0, 0, 255), 3)

            prev_center = center
            prev_point = point

        cv2.imshow("Detected Circle", frame)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

if angular_speed_list:
    avg_angular_speed = np.median(angular_speed_list)
    print(f"Average Angular Speed: {avg_angular_speed:.2f} deg/s")


C:\Users\DELL\AppData\Local\Temp\ipykernel_17464\3690215287.py:6: RuntimeWarning: overflow encountered in scalar subtract
  vector1 = np.array([point1[0] - center[0], point1[1] - center[1]])
C:\Users\DELL\AppData\Local\Temp\ipykernel_17464\3690215287.py:7: RuntimeWarning: overflow encountered in scalar subtract
  vector2 = np.array([point2[0] - center[0], point2[1] - center[1]])


Average Angular Speed: 1349.80 deg/s
